# Stock data analysis

In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Connecting to OpenAI

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [3]:
openai = OpenAI()

In [4]:
from urllib.parse import urlencode
import datetime

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [5]:
class YahooFinanceWebsite:
    def __init__(self, stock_symbol):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.stock_symbol = stock_symbol.upper()

    def __build_url(self, params):
        base_url = f"https://finance.yahoo.com/quote/{self.stock_symbol}/history/"
        query_string = urlencode(params)
        return f"{base_url}?{query_string}"

    def get_stock_data(self):
        datetime_now = datetime.datetime.now()
        datetime_year_ago = datetime_now - datetime.timedelta(days=365)
        params = {"frequency": "1wk", "period1": datetime_year_ago.timestamp(), "period2": datetime_now.timestamp()}
        url = self.__build_url(params)
        response = requests.get(url, headers=headers)

        soup = BeautifulSoup(response.content, 'html.parser')
        
        title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()

        html_table_data = soup.find("table")

        return title, html_table_data

In [6]:
def build_stock_analysis_prompt(stock_symbol, title, stock_table_data):
    sys_prompt = r"""You are an assistant that analyzes the contents of HTML formated table that contains data on a specific stock.
    The HTML table contains the date, open price, close price, low and highs aggregated for every week over one year timeframe.
    Ignoring text, tags or html attributes that might be navigation related. 
    Respond in Markdown format"""
    
    user_prompt = f"The data provided below in the HTML table format for {stock_symbol} from the Yahoo Finances.\
    Make the explaination easy enough for a newbie to understand. \
    Analyze and Summarize the trends on this stock:\n{stock_table_data}\n\n\
    Also, calculate the total returns in percentage one could have expected over this period."
    
    return [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt}
    ]

In [7]:
def analyze_stock_trends(stock_symbol):
    stock_data_page = YahooFinanceWebsite(stock_symbol)
    title, stock_table_data = stock_data_page.get_stock_data()
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = build_stock_analysis_prompt(stock_symbol, title, stock_table_data)
    )
    return response.choices[0].message.content

def display_analysis(stock_symbol):
    display(Markdown(analyze_stock_trends(stock_symbol)))

In [8]:
display_analysis("GOOG")

### Summary of Stock Trends for GOOG

Based on the provided weekly closing prices for Google (GOOG) over approximately one year, let's analyze the trends and calculate potential returns.

#### Key Data Overview
- **Price Movement:** The stock opened at prices as low as **136.54** and reached highs up to **202.14**.
- **Closing Prices:** 
  - The most recent closing price (January 13, 2025) was **192.29**.
  - Closing prices earlier in the timeline ranged significantly, including lows around **142.17**.

#### Observed Trends
1. **Upward Trend:**
   - Over the one-year duration (January 2024 to January 2025), GOOG's stock price experienced an overall upward trend, particularly notable between **Jan 2024** (around **143.54**) and **Dec 2024** (up to **194.04**).
   - The stock saw moments of significant growth, especially from **March 2024** (around **137.07**) to **January 2025**.

2. **Volatility:**
   - The stock exhibited volatility, with peaks (highs) frequently exceeding **$200**, indicating that prices can vary greatly even within short spans.
   - There are notable drops (such as the drop from **167.00** in mid-August to **155.72** by mid-April).

3. **Recovery and Growth:**
   - Following significant drops, there were recoveries, such as rising from **160.28** at the end of March to **192.29** in January.
   - The stock frequently showed weekly price rebounds after dips, suggesting a resilient performance through fluctuations.

#### Total Returns Calculation
To calculate the total returns over the year based on the provided closing prices:

- **Start Price (Jan 15, 2024):** **147.97**
- **End Price (Jan 13, 2025):** **192.29**

**Total Return Formula:**
\[ \text{Total Return (\%)} = \left( \frac{\text{End Price} - \text{Start Price}}{\text{Start Price}} \right) \times 100 \]

**Substituting the Values:**
\[ \text{Total Return (\%)} = \left( \frac{192.29 - 147.97}{147.97} \right) \times 100 \]

Calculating:
\[ = \left( \frac{44.32}{147.97} \right) \times 100 \]
\[ = 29.96\% \]

### Conclusion
- Over the analyzed year, GOOG's stock price reflects a **29.96% return** on investment, highlighting a solid performance for potential investors.
- The stock showcases a general upward momentum despite periods of volatility, making it a noteworthy option for consideration in future investments. 

### Investment Insight
For a newbie in the stock market, this analysis suggests that while there are fluctuations in stock prices, consistent upward trends can offer good returns. Diversifying investments and staying informed about market behaviors is essential for successful investing.

In [9]:
display_analysis("PFE")

### Analysis and Summary of PFE (Pfizer Inc.) Stock Trends

#### **Overview of the Table Data:**
The provided data displays weekly stock performance for Pfizer Inc. (PFE) over the past year, covering key metrics such as:
- **Date**: The specific week of the data.
- **Open**: The price at which the stock opened for that week.
- **High**: The highest price during the week.
- **Low**: The lowest price during the week.
- **Close**: The price at which the stock closed for that week.
- **Adj Close**: The adjusted closing price considering dividends and splits.
- **Volume**: The number of shares traded during that week.

#### **Trends Observed:**
1. **Price Fluctuations**:
   - The price of PFE shares fluctuated notably throughout the year. Early in the year, prices ranged approximately from $25 to $30.
   - The stock hit higher points at around $30.93 and lower points around $24.48.
   - A significant amount of fluctuation occurred between $25 and $29 during the second half of the year.

2. **Overall Performance**:
   - The overall trend shows a decline from about January 2024 ($28.60) to lower levels towards the end of the year, reaching as low as $25.40 in late April 2024.
   - After April, the stock fluctuated within the range of $25 to $30 throughout the remaining months of the year.

3. **Volume Insights**:
   - Volume peaked significantly on several occasions, specifically in late December and early January, with trading volumes sometimes exceeding 300 million shares, indicating high investor interest and activity.

4. **Dividend Payments**:
   - The stock pays dividends regularly (notably on dates like Nov 8, Jul 26, and Jan 25), which may influence an investor's decision-making regarding holding or selling the stock due to the additional income derived from dividends.

5. **Final Context**:
   - The last recorded close in January 2025 was $26.80, indicating that while the stock has experienced volatility, it has not significantly strayed from this range.

#### **Total Returns Calculation**:
To calculate the total returns in percentage over the analyzed period:

1. **Start Price**: Closing price on the start date (Jan 15, 2024): $28.28
2. **End Price**: Closing price on the end date (Jan 13, 2025): $26.80
3. **Total Return Formula**:
   \[
   \text{Total Return} = \left(\frac{\text{End Price} - \text{Start Price}}{\text{Start Price}}\right) \times 100
   \]
   \[
   \text{Total Return} = \left(\frac{26.80 - 28.28}{28.28}\right) \times 100 = -5.25\%
   \]

### **Conclusion**:
- Over the analyzed period, PFE stock displayed significant fluctuations with a general downtrend towards the end of the year, concluding with a total return of approximately **-5.25%**. 
- The drastic variations in price, coupled with high trading volumes, suggest that investors should remain informed and vigilant regarding market conditions and company performance to make educated investment decisions.

In [10]:
display_analysis("AAPL")

# AAPL Stock Trends Analysis

Here's a summarized analysis of Apple Inc. (AAPL) stock based on the provided weekly data for 2024 and part of 2025.

## Key Metrics
- **Open Price**: Price of the stock at the beginning of the week.
- **Close Price**: Price of the stock at the end of the week.
- **High and Low**: The highest and lowest prices of the stock during the week.
- **Volume**: The amount of shares traded during the week.

## Observations and Trends

### General Trend Over Time
1. **Increasing Trend**: The stock price shows an overall increasing trend from early 2024 to early 2025. The stock opened at around **$182.16** in mid-January 2024 and reached **$234.40** by mid-January 2025.
   
2. **Resistance and Support Levels**: 
   - **Resistance**: The stock reached its highest point of around **$254.77** in late December 2024. This indicates an area where selling may occur, creating a challenge for the price to rise further.
   - **Support**: In mid-April 2024, the stock dropped to around **$165.00**, showing that this level acted as a support point where buyers were willing to step in.

3. **Volatility**: There were periods of higher volatility, especially between **May and June 2024**, where the weekly closing prices fluctuated significantly (e.g., from around **$196** to **$212**).

4. **Volume Fluctuations**: 
   - Noticeable spikes in volume occurred at times which indicate increased interest from investors, both buying and selling behaviors.
   - For instance, high volumes of above **600 million** shares were observed in early June 2024.

### Dividends
- The stock provided dividends on several occasions during 2024, notably in **November 2024** and **May 2024** with a dividend of **$0.25** and **$0.25**, respectively. This is attractive for income-focused investors.

## Total Returns Calculation

To calculate the total returns based on the opening price on Jan 15, 2024, and the closing price on Jan 13, 2025:

- **Opening Price (Jan 15, 2024)**: **$182.16**
- **Closing Price (Jan 13, 2025)**: **$234.40**

### Total Returns Formula
\[
\text{Total Returns (\%)} = \left(\frac{\text{Final Price} - \text{Initial Price}}{\text{Initial Price}}\right) \times 100
\]

### Calculation
\[
\text{Total Returns (\%)} = \left(\frac{234.40 - 182.16}{182.16}\right) \times 100 \approx 28.66\%
\]

## Summary
- The AAPL stock price has shown a strong upward trend from early 2024 to early 2025, with some fluctuations in price.
- Key points of resistance and support indicate areas where the stock price encounters higher selling or buying interests.
- The stock issued dividends, contributing to overall shareholder returns.
- An investor from mid-January 2024 to mid-January 2025 could have expected an approximate total return of **28.66%**. 

This analysis indicates a period of growth for AAPL, making it an interesting prospect for potential investors, though caution should be taken during times of volatility.